In [5]:
%%capture
!pip install efficientnet_pytorch
!pip install torch_optimizer

In [6]:
import io
import random
import os
import math
import timm
from PIL import Image
from tqdm import tqdm
import gc
import pandas as pd
import multiprocessing
import numpy as np

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
from torchvision import transforms
from torch.utils.data import DataLoader
from efficientnet_pytorch import EfficientNet
from torchvision.models import efficientnet_v2_s, efficientnet_v2_m, efficientnet_v2_l

from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import f1_score
import albumentations as A
from albumentations.pytorch import ToTensorV2
import cv2


from torch.cuda.amp import GradScaler

In [7]:
# Constants
BATCH_SIZE = 16
GRADIENT_ACCUMULATION_STEPS = 2
NUM_WORKERS = 2
IMAGE_SIZE = 320 
PIN_MEMORY = True 
PATIENCE = 5
N_FOLDS = 5

In [8]:
def calculate_dataset_stats(dataframe, image_dir):
    """Calculate mean and std of the dataset"""
    print("Calculating dataset mean and std...")
    
    # Basic transforms just for stats calculation
    basic_transforms = transforms.Compose([
        transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
        transforms.ToTensor()
    ])
    
    class StatsDataset(torch.utils.data.Dataset):
        def __init__(self, df, img_dir, transform):
            self.df = df
            self.img_dir = img_dir
            self.transform = transform
        
        def __len__(self):
            return len(self.df)
        
        def __getitem__(self, idx):
            img_path = os.path.join(self.img_dir, self.df.iloc[idx].filename)
            image = Image.open(img_path).convert('RGB')
            return self.transform(image)
    
    # Create dataset and loader for stats calculation
    stats_dataset = StatsDataset(dataframe, image_dir, basic_transforms)
    stats_loader = DataLoader(
        stats_dataset,
        batch_size=32,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY
    )
    
    means = []
    stds = []
    
    # Calculate mean and std
    for batch in tqdm(stats_loader, desc="Calculating dataset statistics"):
        means.append(batch.mean((0,2,3)))
        stds.append(batch.std((0,2,3)))
    
    dataset_mean = torch.stack(means).mean(0)
    dataset_std = torch.stack(stds).mean(0)
    
    print(f"Dataset mean: {dataset_mean}")
    print(f"Dataset std: {dataset_std}")
    
    return dataset_mean, dataset_std

class ImageDataset(torch.utils.data.Dataset):
    def __init__(self, dataframe: pd.DataFrame, image_dir: str, mode: str, mean=None, std=None):
        self.df = dataframe
        self.mode = mode
        self.image_dir = image_dir
        
        # Use calculated stats or ImageNet stats as fallback
        self.mean = mean if mean is not None else [0.485, 0.456, 0.406]
        self.std = std if std is not None else [0.229, 0.224, 0.225]
        
        if self.mode == 'train':
            self.transforms = transforms.Compose([
                transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                transforms.RandomHorizontalFlip(p=0.5),
                transforms.RandomVerticalFlip(p=0.3),
                transforms.RandomRotation(15),
                transforms.ColorJitter(
                    brightness=0.2, 
                    contrast=0.2, 
                    saturation=0.2, 
                    hue=0.1
                ),
                transforms.RandomAffine(
                    degrees=10, 
                    translate=(0.1, 0.1), 
                    scale=(0.9, 1.1)
                ),
                transforms.RandomGrayscale(p=0.1),
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ])
        else:
            self.transforms = transforms.Compose([
                transforms.Resize((IMAGE_SIZE, IMAGE_SIZE)),
                transforms.ToTensor(),
                transforms.Normalize(mean=self.mean, std=self.std)
            ])

    def __len__(self):
        return len(self.df)

    def __getitem__(self, index: int):
        try:
            row = self.df.iloc[index]
            image_path = os.path.join(self.image_dir, row.filename)
            
            try:
                image = Image.open(image_path).convert('RGB')
                image = self.transforms(image)
            except Exception as e:
                print(f"Error loading image {image_path}: {str(e)}")
                raise e

            if self.mode == 'test':
                return {
                    'image': image,
                    'filename': row.filename
                }
            else:
                return {
                    'image': image,
                    'target': row.city_id,
                    'filename': row.filename
                }
        except Exception as e:
            print(f"Error in __getitem__ at index {index}: {str(e)}")
            raise e


In [9]:
import timm
from timm import create_model

class EfficientNetB6Head(nn.Module):
    def __init__(self, num_classes, dropout_rate=0.5):
        super().__init__()
        self.encoder = create_model(
            'tf_efficientnet_b6_ns',
            pretrained=True,
            num_classes=0
        )
        
        # Freeze some early layers
        for name, param in list(self.encoder.named_parameters())[:100]:
            param.requires_grad = False
            
        n_features = self.encoder.num_features
        
        # More gradual reduction in dimensions
        # Added LayerNorm for better regularization
        self.head = nn.Sequential(
            nn.Linear(n_features, 1536),
            nn.LayerNorm(1536),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(1536, 768),
            nn.LayerNorm(768),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(768, 384),
            nn.LayerNorm(384),
            nn.ReLU(inplace=True),
            nn.Dropout(dropout_rate),
            
            nn.Linear(384, num_classes)
        )
        
    def forward(self, x):
        features = self.encoder(x)
        return self.head(features)

In [10]:
def prepare_data(train_df):
    skf = StratifiedKFold(n_splits=N_FOLDS, shuffle=True, random_state=42)
    train_df['fold'] = -1
    
    for fold, (train_idx, val_idx) in enumerate(skf.split(train_df, train_df['city'])):
        train_df.loc[val_idx, 'fold'] = fold
    
    return train_df

def load_data(train_df, test_df, train_dir, test_dir, fold=0, mean=None, std=None):
    print("Preparing data loaders...")
    
    label_encoder = LabelEncoder()
    train_df['city_id'] = label_encoder.fit_transform(train_df['city'])
    num_classes = len(label_encoder.classes_)
    
    train_data = train_df[train_df['fold'] != fold].reset_index(drop=True)
    valid_data = train_df[train_df['fold'] == fold].reset_index(drop=True)
    
    train_dataset = ImageDataset(train_data, train_dir, mode='train', mean=mean, std=std)
    valid_dataset = ImageDataset(valid_data, train_dir, mode='valid', mean=mean, std=std)
    test_dataset = ImageDataset(test_df, test_dir, mode='test', mean=mean, std=std)

    print(f"Train dataset size: {len(train_dataset)}")
    print(f"Validation dataset size: {len(valid_dataset)}")
    print(f"Test dataset size: {len(test_dataset)}")
    
    train_loader = DataLoader(
        train_dataset,
        batch_size=BATCH_SIZE,
        shuffle=True,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY,
        drop_last=True
    )
    
    valid_loader = DataLoader(
        valid_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY
    )
    
    test_loader = DataLoader(
        test_dataset,
        batch_size=BATCH_SIZE,
        shuffle=False,
        num_workers=NUM_WORKERS,
        pin_memory=PIN_MEMORY
    )
    
    return train_loader, valid_loader, test_loader, label_encoder, num_classes

def calculate_macro_f1(preds, targets, num_classes):

    # Tensor'ları numpy array'e çevir
    if torch.is_tensor(preds):
        preds = preds.cpu().numpy()
    if torch.is_tensor(targets):
        targets = targets.cpu().numpy()
    
    # Her şehir için F1 skorunu hesapla
    city_f1_scores = []
    
    for city_idx in range(num_classes):
        # True Positives: Doğru tahmin edilen şehir sayısı
        tp = np.sum((preds == city_idx) & (targets == city_idx))
        
        # False Positives: Yanlış şehir olarak tahmin edilenler
        fp = np.sum((preds == city_idx) & (targets != city_idx))
        
        # False Negatives: Kaçırılan şehir tahminleri
        fn = np.sum((preds != city_idx) & (targets == city_idx))
        
        # Precision hesapla
        precision = tp / (tp + fp) if (tp + fp) > 0 else 0.0
        
        # Recall hesapla
        recall = tp / (tp + fn) if (tp + fn) > 0 else 0.0
        
        # F1 skoru hesapla
        if precision + recall > 0:
            f1 = 2 * (precision * recall) / (precision + recall)
        else:
            f1 = 0.0
        
        city_f1_scores.append(f1)
    
    # Macro F1: Tüm şehirlerin F1 skorlarının ortalaması
    macro_f1 = np.mean(city_f1_scores)
    
    return float(macro_f1)

In [11]:
def train_step(train_loader, model, criterion, optimizer, epoch, scaler):
    model.train()
    running_loss = 0.0
    running_f1 = 0.0
    steps = 0
    
    optimizer.zero_grad(set_to_none=True)
    
    pbar = tqdm(enumerate(train_loader), total=len(train_loader))
    for i, data in pbar:
        images = data['image'].cuda(non_blocking=True)
        targets = data['target'].cuda(non_blocking=True)
        
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, targets)
            loss = loss / GRADIENT_ACCUMULATION_STEPS
        
        # Mixed precision backward pass
        scaler.scale(loss).backward()
        
        # Gradient accumulation
        if (i + 1) % GRADIENT_ACCUMULATION_STEPS == 0:
            scaler.step(optimizer)
            scaler.update()
            optimizer.zero_grad(set_to_none=True)
        
        # Calculate metrics
        with torch.no_grad():
            _, preds = torch.max(outputs, 1)
            f1 = calculate_macro_f1(preds, targets, outputs.size(1))
        
        # Update metrics
        running_loss += loss.item() * GRADIENT_ACCUMULATION_STEPS
        running_f1 += f1
        steps += 1
        
        # Update progress bar
        pbar.set_description(
            f'Epoch {epoch} - Loss: {running_loss/steps:.4f}, F1: {running_f1/steps:.4f}'
        )
        
        # Clear memory
        del images, outputs, loss
        torch.cuda.empty_cache()
    
    return running_f1 / steps

def train_model(train_loader, valid_loader, model, criterion, optimizer, scheduler, scaler):
    """Training loop with early stopping"""
    patience = PATIENCE
    counter = 0
    best_valid_f1 = 0.0
    epoch = 0
    
    while True:
        epoch += 1
        torch.cuda.empty_cache()
        gc.collect()
        
        # Train
        train_f1 = train_step(train_loader, model, criterion, optimizer, epoch, scaler)
        
        # Validate
        valid_loss, valid_f1 = validate(valid_loader, model, criterion)
        
        # Scheduler step
        scheduler.step()
        
        print(f"Epoch {epoch} - Train F1: {train_f1:.4f}, Valid F1: {valid_f1:.4f}")
        
        if valid_f1 > best_valid_f1:
            best_valid_f1 = valid_f1
            counter = 0
            torch.save({
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'scheduler_state_dict': scheduler.state_dict(),  # scheduler'ı da kaydedelim
                'epoch': epoch,
                'best_f1': best_valid_f1,
            }, 'best_model.pth')
            print(f"Saved best model with F1: {best_valid_f1:.4f}")
        else:
            counter += 1
            if counter >= patience:
                print(f"Early stopping at epoch {epoch}")
                break
    
    return best_valid_f1  

@torch.no_grad()
def validate(valid_loader, model, criterion):
    model.eval()
    running_loss = 0.0
    running_f1 = 0.0
    steps = 0
    
    for data in tqdm(valid_loader, desc='Validating'):
        images = data['image'].cuda(non_blocking=True)
        targets = data['target'].cuda(non_blocking=True)
        
        # Updated autocast syntax
        with torch.amp.autocast(device_type='cuda'):
            outputs = model(images)
            loss = criterion(outputs, targets)
        
        _, preds = torch.max(outputs, 1)
        f1 = calculate_macro_f1(preds, targets, outputs.size(1))
        
        running_loss += loss.item()
        running_f1 += f1
        steps += 1
        
        del images, outputs, loss
        torch.cuda.empty_cache()
    
    return running_loss / steps, running_f1 / steps

@torch.no_grad()
def predict(test_loader, model, label_encoder):
    model.eval()
    predictions = []
    filenames = []
    
    for data in tqdm(test_loader):
        images = data['image'].cuda(non_blocking=True)
        
        with torch.cuda.amp.autocast():
            outputs = model(images)
            _, preds = torch.max(outputs, 1)
        
        predictions.extend(label_encoder.inverse_transform(preds.cpu().numpy()))
        filenames.extend(data['filename'])
        
        del images, outputs, preds
        torch.cuda.empty_cache()
    
    return filenames, predictions


In [12]:
def generate_submission(test_loader, model, label_encoder):
    filenames, predictions = predict(test_loader, model, label_encoder)
    
    submission = pd.DataFrame({
        'filename': filenames,
        'city': predictions
    })
    
    submission.to_csv('submission.csv', index=False)
    return submission

In [13]:
if __name__ == '__main__':
    # Kaggle paths
    KAGGLE_INPUT = '/kaggle/input/datathon-ai-qualification-round'
    
    # Load data
    train = pd.read_csv(f'{KAGGLE_INPUT}/train_data.csv')
    test = pd.read_csv(f'{KAGGLE_INPUT}/test.csv')
    
    # Set correct image directories
    train_dir = f'{KAGGLE_INPUT}/train/train'
    test_dir = f'{KAGGLE_INPUT}/test/test'
    
    # Print dataset info
    print("Dataset Information:")
    print(f"Training samples: {len(train)}")
    print(f"Test samples: {len(test)}")
    print("\nSample training data:")
    print(train.head())
    print("\nSample test data:")
    print(test.head())
    
    # Verify paths exist
    for path in [train_dir, test_dir]:
        if not os.path.exists(path):
            raise ValueError(f"Path does not exist: {path}")
    
    # First prepare folds - MOVED BEFORE stats calculation
    train = prepare_data(train)
    
    # Calculate dataset statistics ONLY on training fold
    # Get the data for the training fold (excluding validation data)
    training_fold = 0  # Assuming we're using fold 0 for validation
    train_fold_data = train[train['fold'] != training_fold].reset_index(drop=True)
    
    print(f"Calculating statistics using {len(train_fold_data)} training samples...")
    train_mean, train_std = calculate_dataset_stats(train_fold_data, train_dir)
    
    # Enable memory optimizations
    torch.backends.cudnn.benchmark = False
    torch.backends.cuda.matmul.allow_tf32 = True
    torch.backends.cudnn.allow_tf32 = True
    
    try:
        # Set device
        device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
        print(f"Using device: {device}")
        
        # Initialize mixed precision training
        scaler = torch.amp.GradScaler(device='cuda')
        
        # Train on fold 0
        train_loader, valid_loader, test_loader, label_encoder, num_classes = load_data(
            train, test, train_dir, test_dir, fold=training_fold,
            mean=train_mean.tolist(),
            std=train_std.tolist()
        )
        
        # Initialize model
        model = EfficientNetB6Head(num_classes=num_classes)
        model = model.to(device)
        
        # Optimizer and criterion
        criterion = nn.CrossEntropyLoss(label_smoothing=0.1)
        optimizer = optim.AdamW(
        model.parameters(),
        lr=5e-4,  # Daha düşük learning rate
        weight_decay=0.05,  # Daha yüksek weight decay
        betas=(0.9, 0.999)
    )
    
        # Scheduler
        scheduler = torch.optim.lr_scheduler.CosineAnnealingLR(
        optimizer,
        T_max=30,  # maximum epoch sayısı
        eta_min=1e-6  # minimum learning rate
    )
        
        print("Starting training...")
        # Train model with early stopping
        best_valid_f1 = train_model(
            train_loader, valid_loader, model, 
            criterion, optimizer, scheduler, scaler 
        )
        
        print("Loading best model for submission...")
        # Load best model and generate submission
        checkpoint = torch.load('best_model.pth')
        model.load_state_dict(checkpoint['model_state_dict'])
        
        print("Generating submission file...")
        submission = generate_submission(test_loader, model, label_encoder)
        
        # Save submission
        submission_path = '/kaggle/working/submission.csv'
        submission.to_csv(submission_path, index=False)
        
        print(f"Training completed. Best validation F1: {best_valid_f1:.4f}")
        print(f"Submission saved to: {submission_path}")
        
        # Verify submission format
        print("\nVerifying submission format...")
        if set(submission.columns) != {'filename', 'city'}:
            print("Warning: Submission columns do not match required format!")
        if not all(submission['city'].isin(['Istanbul', 'Ankara', 'Izmir'])):
            print("Warning: Submission contains invalid city names!")
        
    except Exception as e:
        print(f"An error occurred during training: {str(e)}")
        raise
    
    finally:
        # Clean up
        torch.cuda.empty_cache()
        gc.collect()

Dataset Information:
Training samples: 7000
Test samples: 2000

Sample training data:
          filename      city
0  image_10000.jpg  Istanbul
1  image_10001.jpg  Istanbul
2  image_10002.jpg    Ankara
3  image_10003.jpg     Izmir
4  image_10004.jpg    Ankara

Sample test data:
          filename  city
0  image_17000.jpg   NaN
1  image_17001.jpg   NaN
2  image_17002.jpg   NaN
3  image_17003.jpg   NaN
4  image_17004.jpg   NaN
Calculating statistics using 5600 training samples...
Calculating dataset mean and std...


Calculating dataset statistics: 100%|██████████| 175/175 [00:45<00:00,  3.85it/s]
/opt/conda/lib/python3.10/site-packages/timm/models/_factory.py:117: UserWarning: Mapping deprecated model name tf_efficientnet_b6_ns to current tf_efficientnet_b6.ns_jft_in1k.
  model = create_fn(


Dataset mean: tensor([0.5070, 0.5327, 0.5378])
Dataset std: tensor([0.2318, 0.2396, 0.2917])
Using device: cuda
Preparing data loaders...
Train dataset size: 5600
Validation dataset size: 1400
Test dataset size: 2000


model.safetensors:   0%|          | 0.00/173M [00:00<?, ?B/s]

Starting training...


...

Epoch 32 - Train F1: 0.9900, Valid F1: 0.9147


Validating: 100%|██████████| 88/88 [00:12<00:00,  6.95it/s]
/tmp/ipykernel_23/2063660693.py:87: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  checkpoint = torch.load('best_m

Epoch 33 - Train F1: 0.9938, Valid F1: 0.9191
Early stopping at epoch 33
Loading best model for submission...
Generating submission file...


  0%|          | 0/125 [00:00<?, ?it/s]/tmp/ipykernel_23/152104679.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
  1%|          | 1/125 [00:00<01:10,  1.76it/s]/tmp/ipykernel_23/152104679.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
  2%|▏         | 2/125 [00:00<00:39,  3.12it/s]/tmp/ipykernel_23/152104679.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
  2%|▏         | 3/125 [00:00<00:30,  4.05it/s]/tmp/ipykernel_23/152104679.py:128: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast():
  3%|▎         | 4/125 [00:01<00:25,  4.83it/s]/

Training completed. Best validation F1: 0.9192
Submission saved to: /kaggle/working/submission.csv

Verifying submission format...
